In [1]:
from pycbc.catalog import Merger
from pycbc.psd import interpolate, inverse_spectrum_truncation
from pycbc.frame import read_frame
from pycbc.filter import highpass, resample_to_delta_t
from astropy.utils.data import download_file
from pycbc.inference import models, sampler
from pycbc.distributions import Uniform, JointDistribution, SinAngle, UniformSky
import numpy
import numpy as np
import h5py
from pycbc.types.frequencyseries import FrequencySeries
from pycbc.detector import Detector
import matplotlib.pyplot as plt

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [2]:
para_data = h5py.File('./Phys607_project3/src/data/parameter.h5', 'r')
para_data.keys()

<KeysViewHDF5 ['coa_phase', 'dec', 'distance', 'inclination', 'mass1', 'mass2', 'polarization', 'q', 'ra', 'redshift', 'spin1_a', 'spin1_azimuthal', 'spin1_polar', 'spin1x', 'spin1y', 'spin1z', 'spin2_a', 'spin2_azimuthal', 'spin2_polar', 'spin2x', 'spin2y', 'spin2z', 'srcmass1', 'srcmass2', 'total_rate']>

In [3]:
parameters = {
    "mass1": "mass1",
    "mass2": "mass2",
    "spin1x": "spin1x",
    "spin1y": "spin1y",
    "spin1z": "spin1z",
    "spin2x": "spin2x",
    "spin2y": "spin2y",
    "spin2z": "spin2z",
    "coa_phase": "coa_phase",
    "inclination": "inclination",
    "distance": "distance",
    "ra": "ra",
    "dec": "dec",
    "polarization": "polarization"
}

parameters2 = {
    "approximant": "IMRPhenomXPHM",
    #"delta_f": 0.1,
    "f_lower": 10
}

In [4]:
data_dic = {key: para_data[dataset_key][:][0] for key, dataset_key in parameters.items()}

In [5]:
all_parameters = {**data_dic, **parameters2}
all_parameters

{'mass1': 60.22234585403269,
 'mass2': 57.169482517199846,
 'spin1x': -0.2900329586425194,
 'spin1y': -0.08403156551895755,
 'spin1z': 0.04665359681814268,
 'spin2x': -0.18902220102218603,
 'spin2y': 0.07817423823443369,
 'spin2z': -0.25795176714346885,
 'coa_phase': 2.4092341172487513,
 'inclination': 2.7579985692108107,
 'distance': 44168.47968834049,
 'ra': 3.3231447888333756,
 'dec': 0.6229703235736507,
 'polarization': 3.5691292400886723,
 'approximant': 'IMRPhenomXPHM',
 'f_lower': 10}

In [6]:
time = 1187008882.4

In [7]:
variables = ['inclination', 'distance', 'ra', 'dec']

In [8]:
for key in variables:
    del all_parameters[key]

In [9]:
variables.append('tc')

In [10]:
static = all_parameters

In [11]:
prior = JointDistribution(tuple(variables),
                Uniform(
                    inclination=(2, numpy.pi),
                    distance=(20, 50),
                    tc=(time+.02, time+0.05),
                ),
                UniformSky(),   # This is a custom distribution which
                                # expects ra / dec and creates a isotropic distribution
        )

In [12]:
data = h5py.File('./Phys607_project3/src/data/data.h5', 'r')

In [13]:
hp = data['hp'][:]

In [14]:
dim = hp.shape[0]

In [15]:
noise = np.random.normal(0, 1, dim)*10**(-20)

In [16]:
noisy_hp = hp + noise

In [17]:
hc = data['hc'][:]
dim = hc.shape[0]
noise = np.random.normal(0, 1, dim)*10**(-20)
noisy_hc = hc + noise

In [18]:
noisy_series_hp = FrequencySeries(noisy_hp, delta_f=0.1)

In [19]:
data_freq = {'H1': noisy_series_hp}

In [21]:
import copy
model = models.SingleTemplate(variables, copy.deepcopy(data_freq),
                              low_frequency_cutoff={'H1':10},
                              static_params = static,
                              prior = prior,
                              sample_rate = 6553.6,
                              )

In [22]:
smpl = sampler.EmceePTSampler(model, 3, 200, nprocesses=8)
_ = smpl.set_p0()

In [25]:
smpl.run_mcmc(200)

IndexError: index 7779181412097 is out of bounds for axis 0 with size 65536